In [75]:
import pandas as pd
import pyarrow.parquet as pq
from pandas.core.tools.datetimes import to_datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [76]:
dst =  pq.read_table('fhv_tripdata_2021-01.parquet')
df = dst.to_pandas()
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


Q1) Downloading the data:

Read the data for January.How many records are there?

Answer:

1154112 Entries 



In [77]:
# Setting up drop_off_time to datetime using pandas
drop_off_time = pd.to_datetime(df.dropOff_datetime)
drop_off_time.head()

0   2021-01-01 00:44:00
1   2021-01-01 01:07:00
2   2021-01-01 01:51:00
3   2021-01-01 00:21:26
4   2021-01-01 00:53:44
Name: dropOff_datetime, dtype: datetime64[ns]

In [78]:
# Setting up pick_up_time to datetime using pandas
pick_up_time = pd.to_datetime(df.pickup_datetime)
pick_up_time.head()

0   2021-01-01 00:27:00
1   2021-01-01 00:50:00
2   2021-01-01 00:01:00
3   2021-01-01 00:13:09
4   2021-01-01 00:38:31
Name: pickup_datetime, dtype: datetime64[ns]

In [79]:
# Calculating the trip duration 
duration = drop_off_time - pick_up_time
duration 

0         0 days 00:17:00
1         0 days 00:17:00
2         0 days 01:50:00
3         0 days 00:08:17
4         0 days 00:15:13
                ...      
1154107   0 days 00:08:45
1154108   0 days 00:57:36
1154109   0 days 00:16:12
1154110   0 days 00:19:26
1154111   0 days 00:36:00
Length: 1154112, dtype: timedelta64[ns]

In [80]:
df['duration'] = duration
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,0 days 00:17:00
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,0 days 00:17:00
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,0 days 01:50:00
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,0 days 00:08:17
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,0 days 00:15:13


In [81]:
# Convert duration to minutes 
to_minus = df.duration.iloc[0]
print(to_minus.total_seconds()/60, 'minutes') 

17.0 minutes


In [82]:
# Coverting DURATION column to display all time in minutes 
df.duration = df.duration.apply(lambda to_minus: to_minus.total_seconds()/60)
#CAUTION DO NOT RE-RUN. A conversation will occur  

In [83]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013,110.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667


In [84]:
# The average duration a trip takes 
Average = df.duration.mean()
print(Average)

19.167224093791006


Q2) Computing Duration:

What's the average trip duration in January?

Answer:

19.16 min 

Is the average duration in minutes

In [85]:
df.duration.describe(percentiles=[0.95,0.98,0.99])

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
50%      1.340000e+01
95%      4.725000e+01
98%      6.613333e+01
99%      9.030000e+01
max      4.233710e+05
Name: duration, dtype: float64

In [86]:
(df.duration >= 1) & (df.duration <= 60)

0           True
1           True
2          False
3           True
4           True
           ...  
1154107     True
1154108     True
1154109     True
1154110     True
1154111     True
Name: duration, Length: 1154112, dtype: bool

In [87]:
# The average trip that are between 1 min and 60 min
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9616276409915155

In [88]:
df = df[((df.duration >= 1) & (df.duration <= 60))]
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


Data Preparation:

How many records did you drop? That were between 1 min and 60 min

In [89]:
print(1154112 - 1109826 ,': Records dropped ')

44286 : Records dropped 


In [90]:
# Changed all PUlocationID's NaN to -1
df['PUlocationID'] = df.PUlocationID.fillna(-1)

/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/1982443802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PUlocationID'] = df.PUlocationID.fillna(-1)


In [91]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [92]:
(df.PUlocationID ==-1).mean()

0.8352732770722617

Q3) Missing Values:

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NaNs

83% were filled with -1

Q4) One-hot Encoding:

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    - Turn the dataframe into a list of dictionaries

    - Fit a dictonary vectorizer
    
    - Get a feature matrix from it 


In [93]:
df['DOlocationID'] = df.DOlocationID.fillna(1)

/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/3629220058.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DOlocationID'] = df.DOlocationID.fillna(1)


In [94]:
df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000


In [95]:
categorical = ['PUlocationID', 'DOlocationID']
df[categorical] = df[categorical].astype('str')
train_dicts = df[categorical].to_dict(orient = 'records')

/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/1952768235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype('str')


In [96]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(1109826, 524)

Q4) One-hot Encoding:

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

What's the dimensionality of this matrix(The number of columns)

Answer:
524 col

Q5) Training a Model

Now let's use the feature matrix from the previous step to train a model
    
    * Train a plain Linear Regression Model with default parameters

    * Calculate the RMSE of the model on the training data

In [97]:
target = 'duration'
y_train = df[target].values
print(y_train)

[17.         17.          8.28333333 ... 16.2        19.43333333
 36.        ]


In [98]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [99]:
y_pred =  lr.predict(X_train)
mean_squared_error(y_train,y_pred, squared=False)

10.531832629168965

Q5) Training a Model

What's the RMSE on train?

The RMSE is 10.53

Q6) Evaluating the Model

Now let's apply this model to the validation dataset (Feb 2021)


In [100]:
dst2 =  pq.read_table('fhv_tripdata_2021-02.parquet')
df2 = dst2.to_pandas()
df2

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037
...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,NaN,31.0,None,B01717
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,NaN,169.0,None,B01717
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285


In [101]:
def clean_Up(dataframe):
    # Calcutaing the Duration 
    drop_off_time = pd.to_datetime(dataframe.dropOff_datetime)
    pick_up_time = pd.to_datetime(dataframe.pickup_datetime)
    duration = drop_off_time - pick_up_time

    # Filtering the
    dataframe['duration'] = duration
    dataframe.duration = dataframe.duration.apply(lambda to_minus: to_minus.total_seconds()/60)
    dataframe = dataframe[((dataframe.duration >= 1) & (dataframe.duration <= 60))]

    dataframe['PUlocationID'] = dataframe.PUlocationID.fillna(-1)
    dataframe['DOlocationID'] = dataframe.DOlocationID.fillna(1)

    return dataframe
    
df2 = clean_Up(df2)
df2

/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/2926816703.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['PUlocationID'] = dataframe.PUlocationID.fillna(-1)
/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/2926816703.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['DOlocationID'] = dataframe.DOlocationID.fillna(1)


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1.0,225.0,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1.0,31.0,None,B01717,13.533333
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1.0,169.0,None,B01717,11.466667
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,25.383333
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,18.050000


In [102]:
def one_hot_Encoding(dataframe):
    categorical = ['PUlocationID', 'DOlocationID']
    dataframe[categorical] = dataframe[categorical].astype('str')
    train_dicts = dataframe[categorical].to_dict(orient = 'records')
    return train_dicts
    
train_dict = one_hot_Encoding(df2)

/var/folders/2v/sxx6c19d6bjf6t_z_bbjs47c0000gn/T/ipykernel_59855/2947310422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[categorical] = dataframe[categorical].astype('str')


In [103]:
dv = DictVectorizer()
X_tr = dv.fit_transform(train_dict)
X_tr.shape


(990113, 525)

In [105]:
y_tr = df2.duration.values
y_tr.shape

(990113,)

In [106]:
lreg = LinearRegression()
lreg.fit(X_tr,y_tr)


LinearRegression()

Q6) Evaluating the model

What's the RMSE on validation?

In [110]:
y_pred =  lreg.predict(X_tr)
print(y_pred.shape)
RSME =  mean_squared_error(y_tr,y_pred, squared=False)
print('RSME: ',RSME)

(990113,)
RSME:  10.962224458974509
